#Worksheet - 6

Learning Objectives:

By the end of this tutorial, learners should be able to effectively train, evaluate, save, and reuse CNN
models, providing a solid foundation for real-world machine learning projects.

• Understand Model Compilation and Training:

• Evaluate and Test Model Performance:

• Make Predictions and Interpret Results:

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import os


In [2]:
data_dir = "/content/drive/MyDrive/ML and AI/week5/FruitinAmazon/train"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# First: create the datasets
train_ds_raw = image_dataset_from_directory(
    train_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

val_ds_raw = image_dataset_from_directory(
    train_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

class_names = train_ds_raw.class_names
num_classes = len(class_names)

# Then apply prefetching
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds_raw.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds_raw.prefetch(buffer_size=AUTOTUNE)


Found 90 files belonging to 6 classes.
Using 72 files for training.
Found 90 files belonging to 6 classes.
Using 18 files for validation.


In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.1),
  layers.RandomZoom(0.1),
])


In [ ]:
base_model = MobileNetV2(input_shape=(224,224,3),
                         include_top=False,
                         weights='imagenet')
base_model.trainable = False  # Freeze base model

# Build the model
model = models.Sequential([
    data_augmentation,
    layers.Rescaling(1./255),  # Normalize

    base_model,
    layers.GlobalAveragePooling2D(),

    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),

    layers.Dense(num_classes, activation='softmax')

])


In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint("best_model.h5", save_best_only=True)
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=callbacks
)


Epoch 1/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2049 - loss: 2.7489


3/3 ━━━━━━━━━━━━━━━━━━━━ 25s 5s/step - accuracy: 0.2057 - loss: 2.7212 - val_accuracy: 0.2222 - val_loss: 1.7013
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 988ms/step - accuracy: 0.5301 - loss: 1.3536


3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.5295 - loss: 1.3505 - val_accuracy: 0.4444 - val_loss: 1.4184
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 992ms/step - accuracy: 0.5891 - loss: 1.0417


3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.5946 - loss: 1.0350 - val_accuracy: 0.6111 - val_loss: 1.2396
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8409 - loss: 0.6159


3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 3s/step - accuracy: 0.8355 - loss: 0.6149 - val_accuracy: 0.6667 - val_loss: 1.1359
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7697 - loss: 0.7054

...


3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.9575 - loss: 0.1384 - val_accuracy: 0.8333 - val_loss: 0.6425
Epoch 17/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9502 - loss: 0.1204


3/3 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.9488 - loss: 0.1206 - val_accuracy: 0.8333 - val_loss: 0.6182
Epoch 18/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 998ms/step - accuracy: 0.9902 - loss: 0.0600


3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.9891 - loss: 0.0610 - val_accuracy: 0.8333 - val_loss: 0.6021
Epoch 19/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9855 - loss: 0.0828


3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 3s/step - accuracy: 0.9822 - loss: 0.0917 - val_accuracy: 0.8333 - val_loss: 0.5836
Epoch 20/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9508 - loss: 0.1987


3/3 ━━━━━━━━━━━━━━━━━━━━ 12s 4s/step - accuracy: 0.9457 - loss: 0.1943 - val_accuracy: 0.8333 - val_loss: 0.5637


In [ ]:
loss, acc = model.evaluate(val_ds)
print(f"Validation Accuracy: {acc*100:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.8333 - loss: 0.5637   
Validation Accuracy: 83.33%


**Baseline CNN**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Dataset path
train_path = "/content/drive/MyDrive/ML and AI/week5/FruitinAmazon/train"
batch_size = 32
img_size = (224, 224)

# Load datasets without prefetching or augmentation
train_ds = image_dataset_from_directory(
    train_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = image_dataset_from_directory(
    train_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

class_names = train_ds.class_names
num_classes = len(class_names)

# Baseline CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')  # Output layer
])

# Compile
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train
history = model.fit(train_ds, validation_data=val_ds, epochs=10)

# Evaluate
loss, acc = model.evaluate(val_ds)
print(f"Validation Accuracy (Baseline): {acc*100:.2f}%")


Found 90 files belonging to 6 classes.
Using 72 files for training.
Found 90 files belonging to 6 classes.
Using 18 files for validation.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 15s 4s/step - accuracy: 0.1181 - loss: 812.6699 - val_accuracy: 0.1667 - val_loss: 458.0863
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step - accuracy: 0.1311 - loss: 381.3445 - val_accuracy: 0.1111 - val_loss: 9.6219
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.3633 - loss: 7.4071 - val_accuracy: 0.2778 - val_loss: 13.0852
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.4106 - loss: 6.2346 - val_accuracy: 0.0000e+00 - val_loss: 2.2002
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.8455 - loss: 0.7808 - val_accuracy: 0.1667 - val_loss: 2.6923
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.8464 - loss: 0.5182 - val_accuracy: 0.1667 - val_loss: 2.9252
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.9891 - loss: 0.1003 - val_accuracy: 0.3889 - val_loss: 2.4322
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.9891 - loss: 0.0601 - val_accuracy: 0.1667 - val_loss: 5.0811

**With Normalization**

In [ ]:
# CNN with Normalization (Rescaling)
model = models.Sequential([
    layers.Rescaling(1./255, input_shape=(224, 224, 3)),  # Normalize pixels

    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compile
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train
history = model.fit(train_ds, validation_data=val_ds, epochs=10)

# Evaluate
loss, acc = model.evaluate(val_ds)
print(f"Validation Accuracy (Baseline): {acc*100:.2f}%")

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - accuracy: 0.1289 - loss: 3.3162 - val_accuracy: 0.1111 - val_loss: 1.8816
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 12s 4s/step - accuracy: 0.2964 - loss: 1.8462 - val_accuracy: 0.1667 - val_loss: 1.7544
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.3507 - loss: 1.4798 - val_accuracy: 0.1667 - val_loss: 1.9287
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step - accuracy: 0.4440 - loss: 1.2729 - val_accuracy: 0.2778 - val_loss: 1.6425
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.5855 - loss: 1.0214 - val_accuracy: 0.2778 - val_loss: 1.6485
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - accuracy: 0.7361 - loss: 0.8875 - val_accuracy: 0.2222 - val_loss: 1.7699
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.6506 - loss: 0.8553 - val_accuracy: 0.2778 - val_loss: 2.0030
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.8377 - loss: 0.6185 - val_accuracy: 0.3889 - val_loss: 1.5894
Epoch 9/

**Added Data Augmentation**

In [ ]:
# Data Augmentation Block
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

# Model with Augmentation and Normalization
model = models.Sequential([
    data_augmentation,  #Augment before feeding to model
    layers.Rescaling(1./255, input_shape=(224, 224, 3)),

    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compile
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train
history = model.fit(train_ds, validation_data=val_ds, epochs=10)

# Evaluate
loss, acc = model.evaluate(val_ds)
print(f"Validation Accuracy (Baseline): {acc*100:.2f}%")


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 16s 3s/step - accuracy: 0.0729 - loss: 3.8718 - val_accuracy: 0.2222 - val_loss: 2.5071
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step - accuracy: 0.2765 - loss: 2.0901 - val_accuracy: 0.3889 - val_loss: 1.6938
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.3207 - loss: 1.5856 - val_accuracy: 0.1667 - val_loss: 1.7566
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 22s 4s/step - accuracy: 0.4253 - loss: 1.5167 - val_accuracy: 0.2778 - val_loss: 1.6129
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 20s 3s/step - accuracy: 0.5078 - loss: 1.3447 - val_accuracy: 0.2778 - val_loss: 1.4635
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step - accuracy: 0.5629 - loss: 1.1981 - val_accuracy: 0.5000 - val_loss: 1.3609
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.6723 - loss: 1.0184 - val_accuracy: 0.4444 - val_loss: 2.1241
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 21s 4s/step - accuracy: 0.5868 - loss: 1.1002 - val_accuracy: 0.4444 - val_loss: 1.2805
Epoch 9/

**Transfer Learning**

In [ ]:
# Load pre-trained MobileNetV2 model without the top layer (classification part)
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze the base model layers (we won't train them)
base_model.trainable = False

# Build the model with MobileNetV2 as the base
model = models.Sequential([
    data_augmentation,
    layers.Rescaling(1./255),  # Normalize input

    base_model,
    layers.GlobalAveragePooling2D(),

    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),

    layers.Dense(len(class_names), activation='softmax')  # num of fruit classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_ds, validation_data=val_ds, epochs=10)

# Evaluate the model
loss, acc = model.evaluate(val_ds)
print(f"Validation Accuracy with Transfer Learning: {acc*100:.2f}%")


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.2452 - loss: 2.5035 - val_accuracy: 0.5000 - val_loss: 1.5388
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.5421 - loss: 1.3415 - val_accuracy: 0.6111 - val_loss: 1.3012
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.5239 - loss: 1.1410 - val_accuracy: 0.7222 - val_loss: 1.1458
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.6801 - loss: 0.8377 - val_accuracy: 0.7222 - val_loss: 1.0615
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.8307 - loss: 0.3787 - val_accuracy: 0.7778 - val_loss: 0.9880
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.8485 - loss: 0.5272 - val_accuracy: 0.7778 - val_loss: 0.9308
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.9232 - loss: 0.2806 - val_accuracy: 0.7778 - val_loss: 0.8968
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9566 - loss: 0.2397 - val_accuracy: 0.7778 - val_loss: 0.8695
Epoch 9/10
3/

Expected Deliverables:


In [ ]:
# Data Augmentation Block
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

# Model with Augmentation, Normalization, and Dropout
model = models.Sequential([
    data_augmentation,  # Augment before feeding to the model
    layers.Rescaling(1./255, input_shape=(224, 224, 3)),

    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),

    # Dropout to prevent overfitting
    layers.Dropout(0.3),  # Drop 30% of the neurons

    layers.Dense(num_classes, activation='softmax')
])

# Compile
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train
history = model.fit(train_ds, validation_data=val_ds, epochs=10)

# Evaluate
loss, acc = model.evaluate(val_ds)
print(f"Validation Accuracy with Dropout: {acc*100:.2f}%")


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 16s 3s/step - accuracy: 0.2222 - loss: 5.5621 - val_accuracy: 0.2222 - val_loss: 1.9153
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 0.1406 - loss: 2.0942 - val_accuracy: 0.0556 - val_loss: 1.8161
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 12s 3s/step - accuracy: 0.1988 - loss: 1.7856 - val_accuracy: 0.0556 - val_loss: 1.8092
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 21s 4s/step - accuracy: 0.1949 - loss: 1.7667 - val_accuracy: 0.0556 - val_loss: 1.8227
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 21s 3s/step - accuracy: 0.2018 - loss: 1.7188 - val_accuracy: 0.1111 - val_loss: 1.8075
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 0.2856 - loss: 1.6574 - val_accuracy: 0.1111 - val_loss: 1.8136
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 12s 3s/step - accuracy: 0.2873 - loss: 1.5139 - val_accuracy: 0.4444 - val_loss: 1.5381
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 22s 4s/step - accuracy: 0.4145 - loss: 1.4480 - val_accuracy: 0.2222 - val_loss: 1.5934
Epoch 9/